In [14]:
from __future__ import print_function
from ortools.graph import pywrapgraph
import pandas as pd
import numpy as np

In [15]:
df = pd.read_excel('hospital xfers.xlsx')
df

,Pickup Time,Start Location,End Location,start time,end time,Unnamed: 5,Unnamed: 6,Unnamed: 7,From/ To,1,2,3,4
0,07:00:00,2,1,0,2,NaN,NaN,NaN,1.0,0.0,30.0,45.0,75.0
1,07:00:00,1,3,0,3,NaN,NaN,NaN,2.0,30.0,0.0,30.0,60.0
2,07:00:00,3,2,0,2,NaN,NaN,NaN,3.0,45.0,30.0,0.0,45.0
3,07:15:00,2,1,1,3,NaN,NaN,NaN,4.0,75.0,60.0,45.0,0.0
4,07:45:00,3,2,3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,18:30:00,2,4,46,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,18:30:00,3,4,46,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,18:30:00,3,1,46,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,18:30:00,2,3,46,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Define start_nodes, end_nodes, capacities, and unit costs between each pair. 
number_of_car = 10
start_nodes = []
end_nodes = []
capacities = []
unit_costs = []
# Define an array of supplies
supplies = []

In [26]:
#creating a source and sink node:
#source node:
start_nodes.extend([204,204,204,204])
end_nodes.extend([0,1,2,3])

#sink node:
start_nodes.extend([200,201,202,203])
end_nodes.extend([205,205,205,205])

#without transfer patients:
#considering different start and end locations
#start_nodes
for i in range(188):
    for j in range(3):
        start_nodes.append(i)

#end_nodes
node_added = [[5,10,19],[3,5,14],[6,3,9],[13,10,7]]
for i in range(188 * 3):
    end_nodes.append(start_nodes[i+8]+node_added[start_nodes[i+8]%4][i%3])
#considering same start and end locations
#start_nodes
for i in range(200):
    start_nodes.append(i)
    end_nodes.append(i+4)
    
#unit_costs
unit_costs.extend([0]*len(start_nodes))

#capacity and supplies
capacities.extend([number_of_car]*len(start_nodes))
supplies.extend([0]* 206)
supplies[204] = int(number_of_car)
supplies[205] = - int(number_of_car)

In [27]:
#with transfer patients:
for i in range(df.shape[0]):
    start_nodes.append(int((df.loc[i,'Start Location'] - 1) + df.loc[i,'start time']*4))
    end_nodes.append(int((df.loc[i,'End Location'] - 1)+ df.loc[i,'end time']*4))
    capacities.append(1)
    unit_costs.append(-1)


[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

In [28]:
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

for i in range(0, len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                            capacities[i], unit_costs[i])


for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])

if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
    print('Minimum cost:', - min_cost_flow.OptimalCost())
else:
    print('There was an issue with the min cost flow input.')

Minimum cost: 103
